In [7]:
#!pip install fastbook
#!pip install bert_score
#!pip install transformers
#! pip install sentencepiece
#!pip install ohmeow-blurr

In [1]:
from rouge_score import rouge_scorer
from nltk.translate.bleu_score import sentence_bleu
from fastai.text.all import *
from fastai import *
from blurr import *
from transformers import *
import transformers
from blurr.text.data.all import *
from blurr.text.modeling.all import *
import bert_score
from bert_score import score

import re
import numpy as np
import pandas as pd
import gzip
import io
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
import re
from fastbook import load_learner
from tqdm import tqdm
from sklearn.model_selection import train_test_split

/opt/conda/envs/pytorch/lib/python3.9/site-packages/blurr/text/modeling/question_answering.py:31: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  squad_metric = load_metric("squad")
[nltk_data] Downloading package punkt to /home/ec2-user/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/ec2-user/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/ec2-user/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/ec2-user/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [2]:
train = pd.read_csv('articles1.csv')
#train2 = pd.read_csv('articles2.csv')
#train3 = pd.read_csv('articles3.csv')
#train.head()

In [3]:
#train = pd.concat([train1, train2, train3])

In [3]:
len(train)

50000

In [4]:
def proprocessing(sample): #preprocessing
    sample = re.sub(r"http\S+", "", sample) #url
    sample = re.sub('\n','', sample) #/n
    pat = r'[^a-zA-z0-9.,!?/:;\"\'\s]'  #special characters
    sample = re.sub(pat, '', sample)
    return sample

In [5]:
proprocessing(train['content'][16])

'  pages into the   journal found in Dylann S. Roofs car     after the assertions of black inferiority, the lamentations over white powerlessness, the longing for a race war     comes an incongruous declaration. I want state that I am morally opposed to psychology, wrote the young white supremacist who would murder nine black worshipers at Emanuel A. M. E. Church in Charleston, S. C. in June 2015. It is a Jewish invention, and does nothing but invent diseases and tell people they have problems when they dont.  Mr. Roof, who plans to represent himself when the penalty phase of his federal capital trial begins on Tuesday, apparently is devoted enough to that proposition or delusion, as some maintain to stake his life on it. Although a defense based on his psychological capacity might be his best opportunity to avoid execution, he seems steadfastly committed to preventing any public examination of his mental state or background. I will not be calling mental health experts or presenting me

In [6]:
train['cleantext']= train['content'].apply(lambda x:proprocessing(x))

In [15]:
#train.head()

In [7]:
train_texts = train[['cleantext','title']]

In [8]:
train_texts.head()

,cleantext,title
0,"WASHINGTON Congressional Republicans have a new fear when it comes to their health care lawsuit against the Obama administration: They might win. The incoming Trump administration could choose to no longer defend the executive branch against the suit, which challenges the administrations authority to spend billions of dollars on health insurance subsidies for and Americans, handing House Republicans a big victory on issues. But a sudden loss of the disputed subsidies could conceivably cause the health care program to implode, leaving millions of people without access to healt...",House Republicans Fret About Winning Their Health Care Suit - The New York Times
1,"After the bullet shells get counted, the blood dries and the votive candles burn out, people peer down from windows and see crime scenes gone cold: a band of yellow police tape blowing in the breeze. The South Bronx, just across the Harlem River from Manhattan and once shorthand for urban dysfunction, still suffers violence at levels long ago slashed in many other parts of New York City. And yet the citys efforts to fight it remain splintered, underfunded and burdened by scandal. In the 40th Precinct, at the southern tip of the Bronx, as in other poor, minority neighborhoods across the c...",Rift Between Officers and Residents as Killings Persist in South Bronx - The New York Times
2,"When Walt Disneys Bambi opened in 1942, critics praised its spare, haunting visual style, vastly different from anything Disney had done before. But what they did not know was that the films striking appearance had been created by a Chinese immigrant artist, who took as his inspiration the landscape paintings of the Song dynasty. The extent of his contribution to Bambi, which remains a mark for film animation, would not be widely known for decades. Like the films title character, the artist, Tyrus Wong, weathered irrevocable separation from his mother and, in the hope of making a lif...","Tyrus Wong, ‘Bambi’ Artist Thwarted by Racial Bias, Dies at 106 - The New York Times"
3,"Death may be the great equalizer, but it isnt necessarily evenhanded. Of all the fields of endeavor that suffered mortal losses in 2016 consider Muhammad Ali and Arnold Palmer in sports and the Hollywood deaths of Carrie Fisher and Debbie Reynolds the pop music world had, hands down, the bleakest year. Start with David Bowie, whose stage persona androgynous glam rocker, dance pop star, electronic experimentalist was as as his music. The year was only days old when the news came that he had died of cancer at 69. He had hinted that his time was short in the lyrics of h...","Among Deaths in 2016, a Heavy Toll in Pop Music - The New York Times"
4,"SEOUL, South Korea North Koreas leader, Kim said on Sunday that his country was making final preparations to conduct its first test of an intercontinental ballistic missile a bold statement less than a month before the inauguration of Donald J. Trump. Although North Korea has conducted five nuclear tests in the last decade and more than 20 ballistic missile tests in 2016 alone, and although it habitually threatens to attack the United States with nuclear weapons, the country has never an intercontinental ballistic missile, or ICBM. In his annual New Years Day speech, which wa...",Kim Jong-un Says North Korea Is Preparing to Test Long-Range Missile - The New York Times


In [9]:
train_texts, test_texts = train_test_split(train_texts, test_size=0.1)
len(train_texts)

45000

In [10]:
pretrained_model_name = "t5-base"
hf_arch, hf_config, hf_tokenizer, hf_model = get_hf_objects(pretrained_model_name, 
                                                                  model_cls=T5ForConditionalGeneration)

# Create mini-batch and define parameters
hf_batch_tfm = Seq2SeqBatchTokenizeTransform(hf_arch, hf_config, hf_tokenizer, hf_model, 
    task='summarization')

# Simple preprocessing
preprocessor = SummarizationPreprocessor(
    hf_tokenizer,
    text_attr='cleantext',
    target_text_attr='title',
    max_input_tok_length=256,
    max_target_tok_length=130,
    min_summary_char_length=30,
)

preprocessed_train = preprocessor.process_df(train_texts)


# Prepare data for training
blocks = (Seq2SeqTextBlock(batch_tokenize_tfm=hf_batch_tfm), noop)
dblock = DataBlock(blocks=blocks, get_x=ColReader('cleantext'), get_y=ColReader('title'), splitter=RandomSplitter())
dls = dblock.dataloaders(preprocessed_train, bs = 2)

/opt/conda/envs/pytorch/lib/python3.9/site-packages/transformers/models/t5/tokenization_t5_fast.py:156: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(
/opt/conda/envs/pytorch/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:3546: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the

In [11]:
#Define performance metrics
seq2seq_metrics = {
        'rouge': {
            'compute_kwargs': { 'rouge_types': ["rouge1", "rouge2", "rougeL"], 'use_stemmer': True },
            'returns': ["rouge1", "rouge2", "rougeL"]
        },
        'bertscore': {
            'compute_kwargs': { 'lang': 'fr' },
            'returns': ["precision", "recall", "f1"]}}

#Model
model = BaseModelWrapper(hf_model)
learn_cbs = [BaseModelCallback]
fit_cbs = [Seq2SeqMetricsCallback(custom_metrics=seq2seq_metrics)]

#Specify training
learn = Learner(dls, model,
                opt_func=ranger,loss_func=CrossEntropyLossFlat(),
                cbs=learn_cbs,splitter=partial(blurr_seq2seq_splitter, arch=hf_arch)).to_fp16()

In [12]:
#Create optimizer with default hyper-parameters
learn.create_opt() 
learn.freeze()

#Training
learn.fit_one_cycle(2, lr_max=3e-5, cbs=fit_cbs)

#Exporting model
learn.save('t5_headline_complete.pkl')

epoch,train_loss,valid_loss,rouge1,rouge2,rougeL,bertscore_precision,bertscore_recall,bertscore_f1,time
0,nan,1.903798,0.339534,0.140191,0.306400,0.740245,0.711044,0.724735,1:39:53
1,nan,2.138460,0.318241,0.122060,0.288941,0.741555,0.700308,0.719738,1:41:50


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

/opt/conda/envs/pytorch/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:3546: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
/opt/conda/envs/pytorch/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:3546: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
/opt/conda/envs/pytorch/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:3546: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of 

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


/opt/conda/envs/pytorch/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:3546: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
/opt/conda/envs/pytorch/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:3546: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
/opt/conda/envs/pytorch/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:3546: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of 

...

/opt/conda/envs/pytorch/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:3546: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
/opt/conda/envs/pytorch/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:3546: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
/opt/conda/envs/pytorch/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:3546: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of 

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

/opt/conda/envs/pytorch/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:3546: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
/opt/conda/envs/pytorch/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:3546: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
/opt/conda/envs/pytorch/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:3546: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of 

	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Path('models/t5_headline_complete.pkl.pth')

In [12]:
learner = learn.load('t5_headline_complete.pkl')

In [13]:
learner.export('models/t5_production_headline_complete.pkl')

In [14]:
inf_learn = load_learner('models/t5_production_headline_complete.pkl')

In [14]:
content = ["Hello David, my name is George and I'm currently a MIDS candidate at UC Berkeley.I'm a big fan of your work and my team of data scientists and developers have created a product I think you’d be interested in. Do you know the hardest part about coming up with an elevator pitch? It’s trying to condense the most important parts into a concise number of words. That same difficulty can be found on Reddit, one of the fastest growing social media sites in the world. We’ve analyzed reddit data over a 10-year period and found that nearly 20% of all posts and articles had ineffective or erroneous titles. So, what’s the solution? Our product is a web-based app that takes a user’s post and generates a title that has been evaluated to accurately summarize the content. We're still in the early stages of development and would appreciate any feedback, would you be interested in meeting with my team to talk more through our product?"]
#content = ["I'm a big fan of your work and my team of data scientists and developers have created a product I think you’d be interested in. Do you know the hardest part about coming up with an elevator pitch? It’s trying to condense the most important parts into a concise number of words."]
outputs = learn.blurr_generate(content, early_stopping=False, num_return_sequences=1, \
                               min_length=5, max_length=20)
for idx, o in enumerate(outputs):
    print(f'=== Prediction {idx+1} ===\n{o}\n')

=== Prediction 1 ===
{'generated_texts': 'Reddit: What’s the solution?'}



In [15]:
def f_rscore(reference, candidate): #assuming both are lists, Rouge
    rscores = []
    rscorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)
    for i in range(len(reference)):
        rscore = rscorer.score(reference[i], candidate[i])
        rscores.append(list(rscore.values())[0][2]) # fscore
    return rscores

def f_bscore(reference, candidate): #assuming both are lists, Bertscore
    P, R, F1 = score(candidate, reference, lang="en")
    return F1.tolist()

def make_predictions(test_texts):
    #list(outputs[0].values())
    predictions = []
    for i in tqdm(range(len(test_texts))):
        outputs = learn.blurr_generate(test_texts.iloc[i].cleantext, early_stopping=False, num_return_sequences=1, \
                                   min_length=5, max_length=20)
        predictions.append(list(outputs[0].values())[0])
    return predictions

def avg_rouge(test_texts, predictions):
    score_list = f_rscore(list(test_texts.title), predictions)
    return np.average(score_list)

def avg_bertscore(test_texts, predictions):
    score_list = f_bscore(list(test_texts.title), predictions)
    return np.average(score_list)

In [16]:
test_texts_sub = test_texts.sample(200)

In [17]:
predictions = make_predictions(test_texts_sub)

100%|█████████████████████████████████████████| 200/200 [00:35<00:00,  5.63it/s]


In [18]:
avg_rouge(test_texts_sub, predictions)

0.31120299221494985

In [19]:
avg_bertscore(test_texts_sub, predictions)

0.8871055501699447